# Based on:
- https://www.kaggle.com/code/mgmostafa/cmi-best-single-model-simplified/output
- https://www.kaggle.com/code/ichigoe/lb0-494-with-tabnet

# Notes on Successful approaches:
- The best performing public model uses Ensemble learning, specifically a **Voting Regressor**, which combines predictions from LightGBM, XGBoost and CatBoost. "This approach is beneficial as it leverages the strengths of multiple models, reducing overfitting and improving overall model performance."

In [114]:
import numpy as np
import pandas as pd
import os
import re
from typing import List

from scipy import stats
import torch

from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression
from sklearn.base import clone
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
# from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer, KNNImputer
# from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, cohen_kappa_score
from scipy.stats import randint, uniform
from sklearn.feature_selection import mutual_info_classif
from scipy.optimize import minimize

from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
import matplotlib.cm as cm
import seaborn as sns

from colorama import Fore, Style
from IPython.display import clear_output
import warnings
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
# from lightgbm import LGBMClassifier
# from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE

warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

In [28]:
def quadratic_weighted_kappa(y_true, y_pred):
    y_pred_rounded = np.round(y_pred).clip(0, 3).astype(int)
    return cohen_kappa_score(y_true, y_pred_rounded, weights='quadratic')

In [152]:
def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    
    # Get basic stats for ALL columns except step
    df.drop('step', axis=1, inplace=True)
    basic_stats = df.describe().values.reshape(-1)
    
    # Add derived features while keeping all original columns
    df['hour'] = pd.to_datetime(df['time_of_day']).dt.hour
    df['time_gap'] = pd.to_datetime(df['time_of_day']).diff().dt.total_seconds()
    
    # Define inactivity using all available information
    enmo_threshold = 0.003
    is_inactive = (
        (df['non-wear_flag'] == 1) |
        (df['enmo'] < enmo_threshold) |
        (df['time_gap'] > 5)
    )
    
    additional_features = [
        (~is_inactive).mean(),  # Activity ratio
        df[df['hour'].between(9, 17)]['enmo'].mean() or 0,  # Daytime activity
        df[~df['hour'].between(9, 17)]['enmo'].mean() or 0,  # Non-daytime activity
        df['time_gap'].gt(5).mean()  # Proportion of large gaps
    ]
    
    features = np.concatenate([basic_stats, additional_features])
    return features, filename.split('=')[1]

In [33]:
def preprocess_data(df: pd.DataFrame, is_train: bool = False, encoders: dict = None, return_encoders: bool = False) -> pd.DataFrame:
    """Preprocess data including encoding and imputation"""
    df = df.copy()
    
    # First handle missing values
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    for col in numerical_cols:
        df[col] = df[col].fillna(df[col].median())
        
    # Identify categorical columns (including seasons)
    categorical_cols = df.select_dtypes(include=['object']).columns
    
    # Handle missing categoricals
    for col in categorical_cols:
        df[col] = df[col].fillna(df[col].mode()[0])
    
    # Encode categorical variables
    if is_train:
        # For training data, create new encoders
        encoders = {}
        for col in categorical_cols:
            if col != 'id':  # Skip ID column
                encoders[col] = LabelEncoder()
                df[col] = encoders[col].fit_transform(df[col])
        # Decide whether to return encoders
        return (df, encoders) if return_encoders else df
    else:
        if encoders is None:
            raise ValueError("Must provide encoders when preprocessing test data")
        # For test data, use provided encoders
        for col in categorical_cols:
            if col != 'id' and col in encoders:  # Skip ID and handle only columns present in train
                df[col] = encoders[col].transform(df[col])
        return df

In [57]:
def feature_engineering(df):
    season_cols = [col for col in df.columns if 'Season' in col]
    df = df.drop(season_cols, axis=1) 
    df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
    df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
    df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
    df['BFP_BMI'] = df['BIA-BIA_Fat'] / df['BIA-BIA_BMI']
    df['FFMI_BFP'] = df['BIA-BIA_FFMI'] / df['BIA-BIA_Fat']
    df['FMI_BFP'] = df['BIA-BIA_FMI'] / df['BIA-BIA_Fat']
    df['LST_TBW'] = df['BIA-BIA_LST'] / df['BIA-BIA_TBW']
    df['BFP_BMR'] = df['BIA-BIA_Fat'] * df['BIA-BIA_BMR']
    df['BFP_DEE'] = df['BIA-BIA_Fat'] * df['BIA-BIA_DEE']
    df['BMR_Weight'] = df['BIA-BIA_BMR'] / df['Physical-Weight']
    df['DEE_Weight'] = df['BIA-BIA_DEE'] / df['Physical-Weight']
    df['SMM_Height'] = df['BIA-BIA_SMM'] / df['Physical-Height']
    df['Muscle_to_Fat'] = df['BIA-BIA_SMM'] / df['BIA-BIA_FMI']
    df['Hydration_Status'] = df['BIA-BIA_TBW'] / df['Physical-Weight']
    df['ICW_TBW'] = df['BIA-BIA_ICW'] / df['BIA-BIA_TBW']
    
    return df

In [35]:
def find_correlated_features(X, threshold=0.95):
    """Identifies correlated features to remove based on training data"""
    corr_matrix = X.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    # Find features to drop
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    print(f"Identified {len(to_drop)} correlated features to remove")
    return to_drop

def remove_correlated_features(X, features_to_drop):
    """Removes specified features from a dataset"""
    X_reduced = X.drop(columns=features_to_drop)
    return X_reduced

# Build Model

In [130]:
class_weights = {
    0: 1.0,
    1: 4.0,  
    2: 8.0,  
    3: 32.0  
}

In [153]:
def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))
def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)


In [38]:
def get_device_params():
    # if os.environ.get('KAGGLE_IS_KAGGLE_GPU_ENABLED', '') == 'True':
    if torch.cuda.is_available():
        XGB_device = {'tree_method': 'gpu_hist'}
        CB_device = {'task_type': 'GPU'}
        print("GPU detected - using GPU accelerated training")
    else:
        XGB_device = {'tree_method': 'hist'}
        CB_device = {'task_type': 'CPU'}
        print("No GPU detected - using CPU training")
    return XGB_device, CB_device

In [124]:
xgb_device, cb_device = get_device_params()

SEED = 42
n_splits = 5

LGB_Params = {
    'learning_rate': 0.07, 
    'random_state': SEED, 
    'n_estimators': 200,
    
    'max_depth': 8, 
    'num_leaves': 300, 
    'min_data_in_leaf': 17,
    # 'max_depth': 6,  # Reduced from 8
    # 'min_data_in_leaf': 10,  # Reduced from 17
    # 'num_leaves': 150,  # Reduced from 300
    
    'feature_fraction': 0.7689, 
    'bagging_fraction': 0.6879, 
    'bagging_freq': 2, 
    'lambda_l1': 4.74, 
    'lambda_l2': 4.743e-06,
    'verbose': -1,
    'class_weight': class_weights,
    'min_child_samples': 5
}

XGB_Params = {
    'learning_rate': 0.05,
    'max_depth': 6,
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 1,
    'reg_lambda': 5,
    'random_state': SEED,
    **xgb_device  # This will add either GPU or CPU parameters
}


CatBoost_Params = {
    'learning_rate': 0.05,
    'depth': 6,
    'iterations': 200,
    'random_seed': SEED,
    'verbose': 0,
    'l2_leaf_reg': 10,
    **cb_device  # This will add either GPU or CPU parameters

}

No GPU detected - using CPU training


In [125]:
models = {
    'lgbm': LGBMRegressor(**LGB_Params),
    'xgboost': XGBRegressor(**XGB_Params),
    'catboost': CatBoostRegressor(**CatBoost_Params)
}

In [154]:
def analyze_predictions_distribution(y_true, predictions, name=""):
    print(f"\n{'-'*20} {name} Prediction Analysis {'-'*20}")
    
    # Get distribution of predictions
    pred_dist = pd.Series(predictions).value_counts(normalize=True).sort_index() * 100
    
    if y_true is not None:
        # Get distribution of true values
        true_dist = pd.Series(y_true).value_counts(normalize=True).sort_index() * 100
        
        print("\nClass Distribution Comparison:")
        comparison_df = pd.DataFrame({
            'True %': true_dist,
            'Predicted %': pred_dist
        }).round(2)
        print(comparison_df)
        
        # Confusion matrix for detailed analysis
        conf_matrix = pd.crosstab(y_true, predictions, 
                                normalize=True) * 100
        print("\nConfusion Matrix (% of true classes predicted as each class):")
        print(conf_matrix.round(2))
        
        # Analysis of severe cases (class 3)
        if 3 in y_true:
            severe_mask = y_true == 3
            severe_predictions = predictions[severe_mask]
            print(f"\nDetailed Analysis of Severe Cases (Class 3):")
            print(f"Total Severe Cases: {sum(severe_mask)}")
            print("Predicted as:")
            for pred_class in range(4):
                count = sum(severe_predictions == pred_class)
                percent = (count / sum(severe_mask)) * 100
                print(f"Class {pred_class}: {count} cases ({percent:.2f}%)")
    else:
        print("\nPrediction Distribution:")
        for class_val, percentage in pred_dist.items():
            print(f"Class {class_val}: {percentage:.2f}%")

In [155]:
# WITH SMOTE
def TrainML(model_class, test_data):

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        # print("Training fold class distribution:")
        # print(y_train.value_counts(normalize=True).sort_index() * 100)
        # print("\nRaw counts:")
        # print(y_train.value_counts().sort_index())

        # smote = SMOTE(random_state=SEED) # basic smote
        
        # aggresive smote (matching distribution more closely)
        class_counts = y_train.value_counts().sort_index()
        majority_count = class_counts[0]  # count of class 0
        
        smote = SMOTE(
            random_state=SEED,
            sampling_strategy={
                1: int(majority_count * 0.3),  # 30% of majority class
                2: int(majority_count * 0.2),  # 20% of majority class
                3: int(majority_count * 0.1)   # 10% of majority class
            }
        )


                      
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

        X_train_resampled = pd.DataFrame(X_train_resampled, columns=X_train.columns)
        
        model = clone(model_class)
        # model.fit(X_train, y_train)
        model.fit(X_train_resampled, y_train_resampled)

        # y_train_pred = model.predict(X_train)
        # y_val_pred = model.predict(X_val)
        y_train_pred = np.clip(model.predict(X_train), 0, 3)
        y_val_pred = np.clip(model.predict(X_val), 0, 3)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        # test_preds[:, fold] = model.predict(test_data)
        test_preds[:, fold] = np.clip(model.predict(test_data), 0, 3)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    analyze_predictions_distribution(y, oof_rounded, "OOF")

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              # x0=[0.3, 1.0, 2.0], args=(y, oof_non_rounded), # to increase prob of predicting class 3
                              method='Nelder-Mead') # Nelder-Mead | # Powell
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)

    # analyze_predictions_distribution(None, tpTuned, "Ensemble Test")
    
    submission = pd.DataFrame({
        'id': test_id,     #sample['id'],
        'sii': tpTuned
    })

    return submission, model


# Steps for improvement:
- Address class imbalance
    - Modify your StratifiedKFold implementation to ensure the rare class (severity 3) is properly represented in each fold:
    - Add sample weights to give more importance to minority classes:
    - Modify your threshold optimization to be more sensitive to minority classes:
    - Adjust model parameters for imbalanced data:

- Add more new features (look at what other contestants are doing)
- Modify StatifiedKFold to use group-based splitting 
- Consider making model weights adaptive based on validation performance
- Make threshold rounder more robust
- Correlation threshold of 0.95 might be too high

In [99]:
# Load data
print("Loading data...")
base_path = '/kaggle/input/child-mind-institute-problematic-internet-use'
train_parquet_path = f'{base_path}/series_train.parquet'
test_parquet_path = f'{base_path}/series_test.parquet'

# Load data
train = pd.read_csv(f'{base_path}/train.csv')
test = pd.read_csv(f'{base_path}/test.csv')

# Get supervised data (rows with non-null target)
# train = train[train['sii'].notna()].copy()
print(f"Train shape: {train.shape}, Test shape: {test.shape}")

Loading data...
Train shape: (3960, 82), Test shape: (20, 59)


In [100]:
# # !ls
# print(os.getcwd())

In [101]:
# Load parquet features with error handling
print("\nProcessing parquet files...")
try:
    print("Loading and processing training data...")
    train_ts = load_time_series(train_parquet_path)
    print("\nShape of training features:", train_ts.shape)
    
    print("\nLoading and processing test data...")
    test_ts = load_time_series(test_parquet_path)
    print("\nShape of test features:", test_ts.shape)
    
    # Save features
    print("\nSaving features...")
    train_ts.to_parquet("actigraphy_features_train.parquet")
    test_ts.to_parquet("actigraphy_features_test.parquet")
except Exception as e:
    print(f"Error processing parquet files: {str(e)}")
    raise

In [102]:
print("\nLoading parquet data...")
try:
    train_ts = pd.read_parquet("actigraphy_features_train.parquet")
    test_ts = pd.read_parquet("actigraphy_features_test.parquet")
except Exception as e:
    print(f"Error in merging: {str(e)}")
    raise


Loading parquet data...


In [103]:
# Merge with original data
print("\nMerging parquet data with CSV data...")
try:
    train = pd.merge(train, train_ts, how="left", on='id')
    test = pd.merge(test, test_ts, how="left", on='id')

    print("\nInitial shapes:")
    print("Train combined:", train.shape)
    print("Test combined:", test.shape)
except Exception as e:
    print(f"Error in merging: {str(e)}")
    raise


Merging parquet data with CSV data...

Initial shapes:
Train combined: (3960, 182)
Test combined: (20, 159)


In [104]:
# Impute using KNN (need to undo line where I remove missing target rows)
imputer = KNNImputer(n_neighbors=5)
numeric_cols = train.select_dtypes(include=['float64', 'int64']).columns
imputed_data = imputer.fit_transform(train[numeric_cols])
train_imputed = pd.DataFrame(imputed_data, columns=numeric_cols)
train_imputed['sii'] = train_imputed['sii'].round().astype(int)
for col in train.columns:
    if col not in numeric_cols:
        train_imputed[col] = train[col]

In [105]:
# Preprocess with error handling
print("\nPreprocessing data...")
try:
    train, encoders = preprocess_data(train, is_train=True, return_encoders=True)
    test = preprocess_data(test, is_train=False, encoders=encoders)
except Exception as e:
    print(f"Error in preprocessing: {str(e)}")
    raise


Preprocessing data...


In [106]:
train = feature_engineering(train)
train = train.dropna(thresh=10, axis=0) # only keep rows with at least 10 Non-null
test = feature_engineering(test)

In [107]:
print("\nLast Prep for Training...")
try:
    exclude_patterns = ['id', 'PCIAT', 'sii']
    feature_cols = [col for col in train.columns 
                   if not any(pattern in col for pattern in exclude_patterns)]
    X = train[feature_cols]
    y = train['sii']
    features_to_drop = find_correlated_features(X, threshold=0.95) 
    X = remove_correlated_features(X, features_to_drop)
    
    test_id = test['id'].copy()
    test = test[feature_cols]
    test = remove_correlated_features(test, features_to_drop)
except Exception as e:
    print()
    raise


Last Prep for Training...
Identified 47 correlated features to remove


In [150]:
# Train models
print("\nTraining models...")
try:
    models = LGBMRegressor(**LGB_Params)
    # models = {
        # 'lgbm': LGBMRegressor(**LGB_Params),
        # 'xgboost': XGBRegressor(**XGB_Params),
        # 'catboost': CatBoostRegressor(**CatBoost_Params)
    # }
    # ensemble_submission, scores, weights = weighted_ensemble_train(X, y, test_features, models)
    submission, trained_model = TrainML(models, test)
except Exception as e:
    print(f"Error in model training: {str(e)}")
    raise

# Save submission
print("\nSaving submission...")
# ensemble_submission.to_csv('submission.csv', index=False)
submission.to_csv('submission.csv', index=False)

print("Done!")

Training Folds: 100%|██████████| 5/5 [00:09<00:00,  1.96s/it]


-------------------- OOF Prediction Analysis --------------------

Class Distribution Comparison:
     True %  Predicted %
0.0   71.16        54.49
1.0   18.43        40.73
2.0    9.55         4.65
3.0    0.86         0.13

Confusion Matrix (% of true classes predicted as each class):
col_0      0      1     2     3
sii                            
0.0    48.66  21.34  1.16  0.00
1.0     4.52  12.30  1.62  0.00
2.0     1.31   6.59  1.52  0.13
3.0     0.00   0.51  0.35  0.00

Detailed Analysis of Severe Cases (Class 3):
Total Severe Cases: 34
Predicted as:
Class 0: 0 cases (0.00%)
Class 1: 20 cases (58.82%)
Class 2: 14 cases (41.18%)
Class 3: 0 cases (0.00%)
Mean Train QWK --> 0.8494
Mean Validation QWK ---> 0.4491


----> || Optimized QWK SCORE ::  0.481

Saving submission...
Done!
